# Scikit-Learn binary classification model.  Remote train via Azure ML Compute (AML Cluster) 
## and optional HyperDrive (hyper-parameter tunning with multiple child runs) 

This notebook showcases the creation of a ScikitLearn Binary classification model by remotely training on Azure ML Compute Target (AMLCompute Cluster). It shows multiple ways of remote training like using a single Estimator, a ScriptRunConfig and hyper-parameter tunning with HyperDrive with multiple child trainings

## Check library versions
This is important when interacting with different executions between remote compute environments (cluster) and the instance/VM with the Jupyter Notebook.
If not using the same versions you can have issues when creating .pkl files in the cluster and downloading them to load it in the Jupyter notebook.

In [10]:
# Check versions
import azureml.core
import sklearn
import joblib
import pandas 

print("Azure SDK version:", azureml.core.VERSION)
print('scikit-learn version: {}.'.format(sklearn.__version__))
print('joblib version: {}.'.format(joblib.__version__))
print('pandas version: {}.'.format(pandas.__version__))

Azure SDK version: 1.19.0
scikit-learn version: 0.22.2.post1.
joblib version: 0.14.1.
pandas version: 0.25.3.


## Setup and connect to AML Workspace

In [11]:
from azureml.core import Workspace

#ws = Workspace.from_config('../../../config.json') # if you run this locally download config.json and place it in root folder of the repo
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, sep='\n')

amlworkspacesahiep2141
MCW_Synapse
westeurope


## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [12]:
from azureml.core import Experiment

experiment_name = 'amlcompute-train-german-credit'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to Azure ML Compute

Azure ML supports a range of compute targets, which you can define in your workpace and use to run experiments; paying for the resources only when using them.

### Create project directory and copy the training script into the project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [13]:
import os
import shutil
project_folder = './amlcompute-train-german-credit'
os.makedirs(project_folder, exist_ok=True)

# Copy the training script into the project directory
shutil.copy('train.py', project_folder)

'./amlcompute-train-german-credit/train.py'

### Connect or Create a Remote AML compute cluster

Try to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`).
If not available, create a new cluster from the code.

In [14]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create an AzureMl Compute resource (a container cluster)
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           vm_priority='dedicated', 
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Fetch the AML Dataset

In [15]:
aml_dataset = ws.datasets['german-credit']

## Create Environment 
#### Optionally list all the available environments and packages in your AML Workspace

In [17]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

# List Environments and packages in my workspace
for env in envs:
    if env.startswith("AzureML"):
        print("Name", env)
        #print("packages", envs[env].python.conda_dependencies.serialize_to_string())
        
# Use curated environment from AML named "AzureML-Tutorial"
curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")  

# Custom environment: 
# Environment.get(workspace=ws, name="myenv", version="1")

Name AzureML-Minimal
Name AzureML-Tutorial
Name AzureML-TensorFlow-2.2-GPU
Name AzureML-TensorFlow-2.2-CPU
Name AzureML-PyTorch-1.6-CPU
Name AzureML-PyTorch-1.6-GPU
Name AzureML-Triton
Name AzureML-TensorFlow-2.3-CPU
Name AzureML-TensorFlow-2.3-GPU
Name AzureML-DeepSpeed-0.3-GPU
Name AzureML-XGBoost-0.9-Inference-CPU
Name AzureML-PyTorch-1.6-Inference-CPU
Name AzureML-Minimal-Inference-CPU
Name AzureML-TensorFlow-1.15-Inference-CPU
Name AzureML-PyTorch-1.5-GPU
Name AzureML-PyTorch-1.5-CPU
Name AzureML-PyTorch-1.3-CPU
Name AzureML-VowpalWabbit-8.8.0
Name AzureML-AutoML-GPU
Name AzureML-Designer-Score


## Configure & Run

## Option A 1. Configure & Run using ScriptRunConfig & Environment 
(Easiest path using curated environments)

In [18]:
# Add training script to run config
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment

script_runconfig = ScriptRunConfig(source_directory=project_folder, 
                                   script="train.py",
                                   arguments=[aml_dataset.as_named_input('credit')]
                                  )

# Attach compute target to run config
script_runconfig.run_config.target = amlcompute_cluster_name
# runconfig.run_config.target = "local"

# Attach environment to run config
script_runconfig.run_config.environment = curated_environment

### Option A 2. Run the experiment with a single ScriptRunConfig and single run (Optional Jump)
Jump this step if you want to use HyperDrive (Go to Option C)

In [19]:
# Submit the Experiment Run to the AML Compute 
run = experiment.submit(script_runconfig)
run

Experiment,Id,Type,Status,Details Page,Docs Page
amlcompute-train-german-credit,amlcompute-train-german-credit_1617818669_13868741,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


### Option A 3. Monitor Run 

In [20]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

#### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use wait_for_completion to show when the model training is finished:

In [21]:
run.wait_for_completion(show_output=True)  # specify True for a verbose log

RunId: amlcompute-train-german-credit_1617818669_13868741
Web View: https://ml.azure.com/experiments/amlcompute-train-german-credit/runs/amlcompute-train-german-credit_1617818669_13868741?wsid=/subscriptions/7d48758f-d40b-4252-854c-e7d8f2ed7645/resourcegroups/MCW_Synapse/workspaces/amlworkspacesahiep2141

Streaming azureml-logs/55_azureml-execution-tvmps_38042687d40e575a0eb93ea4bd4be2470034017d299f7ce33297e243302fb59e_d.txt

2021-04-07T18:08:32Z Starting output-watcher...
2021-04-07T18:08:32Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-04-07T18:08:32Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-04-07T18:08:32Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_6ea7d39519e353893a72f37ef0f2e906
4007a89234b4: Pulling fs layer
5dfa26c6b9c9: Pulling fs layer
0ba7bf18aa40: Pulling fs layer
4c6ec688ebe3: Pulling fs layer
574f361512d6: Pulling fs layer
db4d1e2d7079: Pul

{'runId': 'amlcompute-train-german-credit_1617818669_13868741',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-07T18:08:32.195886Z',
 'endTimeUtc': '2021-04-07T18:11:09.894624Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '9715ca54-6642-49ff-ae5e-8ee17166526e',
  'azureml.git.repository_uri': 'https://sahieple@dev.azure.com/sahieple/aml_workshop/_git/aml_workshop',
  'mlflow.source.git.repoURL': 'https://sahieple@dev.azure.com/sahieple/aml_workshop/_git/aml_workshop',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'b8d00748f0220f31960f5175d9b4ad488dab0f44',
  'mlflow.source.git.commit': 'b8d00748f0220f31960f5175d9b4ad488dab0f44',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'bb19eea4-9340-4ad7-919f-3deb48290e6a'}, 'consumptionDetails':

## Option B Configure an Estimator with Environment

#### Copy the curated environment to a new custom environment and register it into your Workspace
##### You can run this step just once since the custom Environment will be registered in your Workspace after this step. 

Need to copy the curated environment into a custom environment which will be updated by the Estimator.

If you try to directly use a curated environment with an Estimator you'll get the error:
Error: "Environment name can not start with the prefix AzureML. To alter a curated environment first create a copy of it."

##### This a potential PAPER-CUT: 
- Copying a curated environment to a custom environment and registering is not straight forward! You need to save Conda dependecies as a file, load it from there, etc.
- An Estimator should be able to use a curated environment, too.

In [38]:
# (CDLTLL) POTENTIAL PAPER-CUT: Copying a curated environment to a custom environment and registering is not straight forward!

# Save curated environment definition to folder (Two files, one for conda_dependencies.yml and another file for azureml_environment.json)
curated_environment.save_to_directory(path="./curated_environment_definition", overwrite=True)

# Create custom Environment from Conda specification file
custom_environment = Environment.from_conda_specification(name="custom-workshop-environment",
                                                          file_path="./curated_environment_definition/conda_dependencies.yml")

# Save curated environment definition to folder (Two files, one for conda_dependencies.yml and another file for azureml_environment.json)
custom_environment.save_to_directory(path="./custom_environment_definition", overwrite=True)

custom_environment.register(ws)

envs = Environment.list(workspace=ws)

# List Environments and packages in my workspace
for env in envs:
    if env.startswith("custom"):
        print("Environment Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())


# Use Estimator with Custom Environment 
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn

# Need to use custom environment with an Estimator as the Estimator will updated it when creating a Docker Image even if you don't have new pip or Conda packages.
# If you try to directly use a curated environment with an Estimator you'll get the error:
# Error: "Environment name can not start with the prefix AzureML. To alter a curated environment first create a copy of it."

# Load Custom Environment from Workspace
custom_environment = Environment.get(workspace=ws, name="custom-workshop-environment")  # ,version="1"
print(custom_environment)

script_params = {
                 '--solver': 'liblinear',
                 '--penalty': 'l2'
                 }

# Using plain Estimator class with custom Environment
estimator = Estimator(source_directory=project_folder, 
                      script_params=script_params,
                      compute_target=aml_remote_compute,
                      # use_docker=True, #AML Cluster only supports Docker runs
                      entry_script='train.py',
                      environment_definition= custom_environment,
                      inputs=[ws.datasets['german-credit'].as_named_input('credit')])

Environment Name custom-workshop-environment
packages channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.25.0
  - azureml-defaults==1.25.0
  - azureml-telemetry==1.25.0
  - azureml-train-restclients-hyperdrive==1.25.0
  - azureml-train-core==1.25.0
  - azureml-widgets==1.25.0
  - azureml-pipeline-core==1.25.0
  - azureml-pipeline-steps==1.25.0
  - azureml-opendatasets==1.25.0
  - azureml-automl-core==1.25.0
  - azureml-automl-runtime==1.25.0.post1
  - azureml-train-automl-client==1.25.0
  - azureml-train-automl-runtime==1.25.0
  - azureml-train-automl==1.25.0
  - azureml-train==1.25.0
  - azureml-sdk==1.25.0
  - azureml-interpret==1.25.0
  - azureml-tensorboard==1.25.0
  - azureml-mlflow==1.25.0
  - mlflow
  - sklearn-pandas
- pandas
- numpy
- tqdm
- scikit-learn
- matplotlib
name: azureml_220813aa74c252741cc887dcbeb01c68

Environment(Name: custom-workshop-environment,
Version: 1)
'Estimator' is deprecated. Please use 'ScriptRunConfig' from 'azur

### Option B) 2) Run the experiment with a single Estimator 

In [39]:
run = experiment.submit(estimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
amlcompute-train-german-credit,amlcompute-train-german-credit_1617795803_47c1d40c,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


### Option B) 3. Monitor the run 

In [40]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).
#### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use wait_for_completion to show when the model training is finished:

In [41]:
run.wait_for_completion(show_output=True)  # specify True for a verbose log

RunId: amlcompute-train-german-credit_1617795803_47c1d40c
Web View: https://ml.azure.com/experiments/amlcompute-train-german-credit/runs/amlcompute-train-german-credit_1617795803_47c1d40c?wsid=/subscriptions/7d48758f-d40b-4252-854c-e7d8f2ed7645/resourcegroups/MCW_Synapse/workspaces/amlworkspacesahiep2141

Streaming azureml-logs/20_image_build_log.txt

2021/04/07 11:43:28 Downloading source code...
2021/04/07 11:43:29 Finished downloading source code
2021/04/07 11:43:30 Creating Docker network: acb_default_network, driver: 'bridge'
2021/04/07 11:43:30 Successfully set up Docker network: acb_default_network
2021/04/07 11:43:30 Setting up Docker configuration...
2021/04/07 11:43:31 Successfully set up Docker configuration
2021/04/07 11:43:31 Logging in to registry: 70a2bbaeccaf4325a42626f1a68d001b.azurecr.io
2021/04/07 11:43:33 Successfully logged into 70a2bbaeccaf4325a42626f1a68d001b.azurecr.io
2021/04/07 11:43:33 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', 

## Option C
### Configure and Run with Intelligent hyperparameter tuning (HyperDrive using Estimator)
IMPORTANT: You need to have created either an Estimator or an ScriptRunConfig in the previous steps (Without submitting the experiment).

The adjustable parameters that govern the training process are referred to as the hyperparameters of the model. The goal of hyperparameter tuning is to search across various hyperparameter configurations and find the configuration that results in the best performance.

To demonstrate how Azure Machine Learning can help you automate the process of hyperarameter tuning, we will launch multiple runs with different values for numbers in the sequence. First let's define the parameter space using random sampling.

### Option C) 1. Create a hyperparameter sweep
First, we will define the hyperparameter space to sweep over. 
In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, Accuracy.

In [ ]:
# from azureml.train.hyperdrive import *
from azureml.train.hyperdrive import RandomParameterSampling, BayesianParameterSampling 
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform
    
# solver{'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, default='lbfgs'
# penalty{'l1', 'l2', 'elasticnet', 'none'}, default='l2' --- Note that some penalty parameters are not supported by some algorithms..
param_sampling = RandomParameterSampling({
                                          "--solver": choice('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
                                          "--penalty": choice('l2')
                                         }
                                        )
# Details on Scikit-Learn LogisticRegression hyper-parameters:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Now we will define an early termination policy. The BanditPolicy basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.


In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
# Note that early termination policy is currently NOT supported with Bayesian sampling
# Check here for recommendations on the multiple policies:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#picking-an-early-termination-policy

Now we are ready to configure a run configuration object, and specify the primary metric 'Accuracy' that's recorded in your training runs. 
If you go back to visit the training script, you will notice that this value is being logged. 
We also want to tell the service that we are looking to maximizing this value. 
We also set the number of samples to 20, and maximal concurrent job to 4.

In [ ]:
# Note that in this case when using HyperDrive, the original Estimator's parameters are not used but the HyperDrive parameters...
hyperdrive_config = HyperDriveConfig(run_config=script_runconfig, 
                                     #estimator=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     max_concurrent_runs=4)

### Option C) 2. Lauch the hyperparameter tuning job.

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Check here how to submit the hyperdrive run as a step of an AML Pipeline:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-parameter-tuning-with-hyperdrive.ipynb

### Option C) 3. Monitor the HyperDrive run

Monitor the progress of the runs with the Jupyter widget.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

In [ ]:
# Get details for debugging:
RunDetails(hyperdrive_run).get_widget_data()

##### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use wait_for_completion to show when the model training is finished:

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

### Option C) 4. Find and get the best model found by HyperDrive¶ 
When all jobs finish, we can find out the one that has the highest accuracy.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
# print(best_run.get_details())

In [ ]:
# Copy 'best_run' to 'run' to re-use the same code also used without HyperDrive
run = best_run

## Display run metrics results
You now have a model trained on a remote cluster. Retrieve the accuracy of the model:

In [ ]:
print(run.get_metrics())

## See files associated with the run

In [ ]:
print(run.get_file_names())

run.download_file('azureml-logs/70_driver_log.txt')

## Download Scikit-Learn model

In [ ]:
# retrieve model for visualization and deployment
# Download the model .pkl file to local (Using the 'run' object)
run.download_file('outputs/model.pkl')

## Register the model
Once you've trained the model, you can save and register it to your workspace. Model registration lets you store and version your models in your workspace to simplify model management and deployment.

Running the following code will register the model to your workspace, and will make it available to reference by name in remote compute contexts or deployment scripts. 

In [ ]:
from azureml.core.model import Model

model_reg = run.register_model(model_name='creditmodel_aml',    # Name of the registered model in your workspace.
                               description='Binary classification model for German credit risk data',
                               model_path='outputs/model.pkl', # Local file to upload and register as a model.
                               model_framework=Model.Framework.SCIKITLEARN,     # Framework used to create the model.
                               model_framework_version='0.20.3',                # Version of scikit-learn used to create the model.
                               tags={'ml-task': "binary-classification", 'business-area': "Risk"},
                               properties={'joblib-version': "0.14.0", 'pandas-version': "0.23.4"},
                               sample_input_dataset=aml_dataset
                              )
model_reg

In [ ]:
# Download Scikit-Learn model pickle file from the model registry
from azureml.core.model import Model
print(Model.get_model_path('creditmodel_aml', _workspace=ws))

model_from_registry = Model(ws,'creditmodel_aml')
model_from_registry.download(target_dir='.', exist_ok=True)

# Try model predictions in this notebook

### Load model into memory

In [ ]:
# Load the model into memory
model = joblib.load('model.pkl')
model

In [ ]:
# Load model and test datasets from .pkl files

# Download the test datasets to local
run.download_file('outputs/x_test.pkl')
run.download_file('outputs/y_test.pkl')

# Load the test datasets into memory
x_test = joblib.load('x_test.pkl')
y_test = joblib.load('y_test.pkl')

# joblib
# https://joblib.readthedocs.io/en/latest/installing.html
# https://joblib.readthedocs.io/en/latest/generated/joblib.load.html

## Make Predictions and calculate Accuracy metric

In [ ]:
from sklearn.metrics import accuracy_score

# Make Multiple Predictions
y_predictions = model.predict(x_test)  # .predict(X[0:1])
y_predictions

print('Accuracy:')
accuracy_score(y_test, y_predictions)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_predictions)
print(cm)

In [ ]:
# Show confusion matrix in a separate window
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# One Prediction
instance_num = 1
# Get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = model.predict(x_test)[instance_num]

print("One Prediction: ")
print(prediction_value)

In [ ]:
predictions = pandas.concat([x_test, y_test], axis=1)
predictions['prediction'] = y_predictions
predictions.head(10)